<a href="https://colab.research.google.com/github/ROARMarketingConcepts/Machine-Learning-Projects/blob/master/Google_PlayStore_Apps_Analysis_SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Google PlayStore Apps Analysis

### Using a Support Vector Machine Classifier (SVM) algorithm to predict the sentiments (Positive, Neutral, Negative) of Google PlayStore App user reviews.

Performed by
Ken Wood

ROAR Marketing Concepts LLC

ken@roarmarketingconcepts.com


### Content

Each app (row) has values for catergory, rating, size, and more.

### Acknowledgements

This information is scraped from the Google Play Store. This app information would not be available without it.

### Inspiration

The Play Store apps data has enormous potential to drive app-making businesses to success. Actionable insights can be drawn for developers to work on and capture the Android market!

### Mount the Google Drive where the datasets are located...

In [1]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


### Install some necessary packages to perform the required analysis...

In [2]:
!pip install -U scikit-learn
!pip install --user --upgrade tables

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime, date
from dateutil.relativedelta import relativedelta

from sklearn.preprocessing import StandardScaler

from math import ceil

from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop

%matplotlib inline

# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

Requirement already up-to-date: scikit-learn in /usr/local/lib/python3.6/dist-packages (0.20.2)
Requirement already up-to-date: tables in /usr/local/lib/python3.6/dist-packages (3.4.4)


Using TensorFlow backend.


### Load the datasets..

In [0]:
reviews = pd.read_csv('/gdrive/My Drive/Colab Notebooks/Google Play Store Apps/google-play-store-apps/googleplaystore_user_reviews.csv')
# store = pd.read_csv('/gdrive/My Drive/Colab Notebooks/Google Play Store Apps/google-play-store-apps/googleplaystore.csv')

### Let's look at some properties of the datasets...

In [4]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64295 entries, 0 to 64294
Data columns (total 5 columns):
App                       64295 non-null object
Translated_Review         37427 non-null object
Sentiment                 37432 non-null object
Sentiment_Polarity        37432 non-null float64
Sentiment_Subjectivity    37432 non-null float64
dtypes: float64(2), object(3)
memory usage: 2.5+ MB


In [5]:
reviews.head()

,App,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity
0,10 Best Foods for You,I like eat delicious food. That's I'm cooking ...,Positive,1.00,0.533333
1,10 Best Foods for You,This help eating healthy exercise regular basis,Positive,0.25,0.288462
2,10 Best Foods for You,NaN,NaN,NaN,NaN
3,10 Best Foods for You,Works great especially going grocery store,Positive,0.40,0.875000
4,10 Best Foods for You,Best idea us,Positive,1.00,0.300000


In [0]:
# store.info()

In [0]:
# store.head()

In [8]:
"""row = store.loc[store['Installs'] == 'Free']
row = row.shift(1,axis='columns')
row['App']=row['Category']
row['Rating'] = row['Reviews']
row['Category'] = np.nan
row['Reviews'] = 19.0
store.loc[store['Installs'] == 'Free'] = row"""

"row = store.loc[store['Installs'] == 'Free']\nrow = row.shift(1,axis='columns')\nrow['App']=row['Category']\nrow['Rating'] = row['Reviews']\nrow['Category'] = np.nan\nrow['Reviews'] = 19.0\nstore.loc[store['Installs'] == 'Free'] = row"

In [0]:
# store.Installs.value_counts()

In [10]:
"""store['Installs'] = store['Installs'].map(lambda x: x.rstrip('+'))
store['Installs'] = store['Installs'].str.replace(',', '').astype(float) 
store['Installs'].head()"""

"store['Installs'] = store['Installs'].map(lambda x: x.rstrip('+'))\nstore['Installs'] = store['Installs'].str.replace(',', '').astype(float) \nstore['Installs'].head()"

### Change the 'Last Updated' column to a pandas datetime variable...

In [0]:
# store['Last Updated'] = store['Last Updated'].apply(lambda x: datetime.strptime(x, '%B %d, %Y'))

In [0]:
# merged = reviews.merge(store, how='left', on = 'App')
# merged.head()

In [13]:
review_sentiment = reviews[['Translated_Review','Sentiment']].copy()
review_sentiment.dropna(inplace=True)
review_sentiment.drop_duplicates(subset = 'Translated_Review',inplace=True )
review_sentiment.Sentiment.value_counts()

Positive    17593
Negative     6240
Neutral      4161
Name: Sentiment, dtype: int64

### Let's code the 'Sentiment' column using sklearn's OrdinalEncoder function...



In [0]:
from sklearn.preprocessing import OrdinalEncoder

sentiment = review_sentiment.Sentiment.values.reshape(-1,1)
ordinal_encoder = OrdinalEncoder()
sentiment_coded = ordinal_encoder.fit_transform(sentiment)
sentiment_coded = sentiment_coded.reshape(27994,)


### Let's do some data cleaning work on the app review text....

In [0]:
import re     # regular expression module

first_text=review_sentiment.Translated_Review[0]
text=re.sub("[^a-zA-Z]"," ",first_text) #changing characters with space
text=text.lower()

In [16]:
print(review_sentiment.Translated_Review[0]) #lets review of changings
print(text)


I like eat delicious food. That's I'm cooking food myself, case "10 Best Foods" helps lot, also "Best Before (Shelf Life)"
i like eat delicious food  that s i m cooking food myself  case     best foods  helps lot  also  best before  shelf life  


### Let's separate each word and then get rid of words that add no value to the review...

In [17]:
import nltk as nlp  # Natural Language ToolKit
nlp.download('punkt')
nlp.download('wordnet')

from nltk.corpus import stopwords

def ConvertTextString(text_string):
  text=nlp.word_tokenize(text_string)         #separate all words
  lemma=nlp.WordNetLemmatizer()
  text=[lemma.lemmatize(i) for i in text]
  text=" ".join(text)
  text_list.append(text)
  return text_list

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [18]:
text_list = []

for i in review_sentiment.Translated_Review:
  ConvertTextString(i)
  
  
text_list[:10]

["I like eat delicious food . That 's I 'm cooking food myself , case `` 10 Best Foods '' help lot , also `` Best Before ( Shelf Life ) ''",
 'This help eating healthy exercise regular basis',
 'Works great especially going grocery store',
 'Best idea u',
 'Best way',
 'Amazing',
 'Looking forward app ,',
 'It helpful site ! It help food get !',
 'good you .',
 'Useful information The amount spelling error question validity information shared . Once fixed , 5 star given .']

### Create a 'bag' of words...

In [0]:
from sklearn.feature_extraction.text import CountVectorizer

max_features=200000
count_vec=CountVectorizer(max_features=max_features,stop_words="english")
sparce_matrix=count_vec.fit_transform(text_list).toarray()
# all_words=count_vec.get_feature_names()
# print("Most used words: ",all_words[2000:2500])


In [22]:
"""from wordcloud import WordCloud

plt.subplots(figsize=(12,12))
wordcloud=WordCloud(background_color="white",width=1024,height=768).generate(" ".join(all_words[2000:2500]))
plt.imshow(wordcloud)
plt.axis("off")
plt.show()"""

'from wordcloud import WordCloud\n\nplt.subplots(figsize=(12,12))\nwordcloud=WordCloud(background_color="white",width=1024,height=768).generate(" ".join(all_words[2000:2500]))\nplt.imshow(wordcloud)\nplt.axis("off")\nplt.show()'

In [23]:
#classification

y = sentiment_coded
x=sparce_matrix

from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)
print(type(x_train),type(x_test),type(y_train),type(y_test))
x_train.shape,x_test.shape,y_train.shape,y_test.shape

<class 'numpy.ndarray'> <class 'numpy.ndarray'> <class 'numpy.ndarray'> <class 'numpy.ndarray'>


((22395, 19345), (5599, 19345), (22395,), (5599,))

### Let's try a Support Vector Machine (SVM) algorithm...we will train on the first 3000 samples to conserve computation time and memory.

In [24]:
from sklearn.svm import SVC

svm_clf = SVC(decision_function_shape="ovr", gamma="auto")
svm_clf.fit(x_train[:3000], y_train[:3000])

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [25]:
print("SVM train set accuracy: ",svm_clf.score(x_train[:3000],y_train[:3000]))

SVM train set accuracy:  0.623


In [26]:
print("SVM test set accuracy: ",svm_clf.score(x_test,y_test))

SVM test set accuracy:  0.6301125200928738


In [0]:
y_pred = svm_clf.predict(x_test)
# y_pred = y_pred.argmax(axis=1)
y_true = y_test

from sklearn.metrics import confusion_matrix
import seaborn as sns

names=["Positive","Negative","Neutral"]
cm=confusion_matrix(y_true,y_pred)
f,ax=plt.subplots(figsize=(5,5))
sns.heatmap(cm,annot=True,linewidth=.5,linecolor="r",fmt=".0f",ax=ax)
plt.xlabel("y_pred")
plt.ylabel("y_true")
ax.set_xticklabels(names)
ax.set_yticklabels(names)
plt.show()

In [0]:
from sklearn.metrics import classification_report
print(classification_report(y_true, y_pred))

### Let's do a GridSearchCV varying the value of C, the penalty parameter. To avoid memory overrun errors, we will do the search using the first 10000 samples of the training set.

In [0]:
from sklearn.model_selection import GridSearchCV
from scipy.stats import randint
from sklearn.metrics import make_scorer,accuracy_score

score = make_scorer(accuracy_score)

param_grid = [{'C': [.01,.025,.05,.075,.1,.25,.5,.75]}]
    

grid_search = GridSearchCV(svm_clf, param_grid,cv=5, scoring=score)
grid_search.fit(x_train[:10000], y_train[:10000])

### Let's find out which value of C provided the best score...

In [0]:
grid_search.best_params_

In [0]:
from sklearn.svm import LinearSVC

svm_clf = LinearSVC(C=0.25, random_state=42)
svm_clf.fit(x_train,y_train)

### We then retrain the Linear SVC model on the training set using C=0.25.

In [0]:
print("Linear SVC train set accuracy: ",svm_clf.score(x_train,y_train))

In [0]:
print("Linear SVC test set accuracy: ",svm_clf.score(x_test,y_test))

### The tuned Linear SVC algorithm provides us with 88.5 percent accuracy on the test set.  This is probably the best we will be able to achieve with a  linearized model.

In [0]:
y_pred = svm_clf.predict(x_test)
# y_pred = y_pred.argmax(axis=1)
y_true = y_test

from sklearn.metrics import confusion_matrix
import seaborn as sns

names=["Positive","Negative","Neutral"]
cm=confusion_matrix(y_true,y_pred)
f,ax=plt.subplots(figsize=(5,5))
sns.heatmap(cm,annot=True,linewidth=.5,linecolor="r",fmt=".0f",ax=ax)
plt.xlabel("y_pred")
plt.ylabel("y_true")
ax.set_xticklabels(names)
ax.set_yticklabels(names)
plt.show()

In [0]:
print(classification_report(y_true, y_pred))